# УЧЕСТЬ ВКЛЮЧЕНИЕ no equality В СТРУКТУРУ

lang должен быть ткой от которого нет no_eq

merge dict запускать не листом а последосвательно к каой паре продумывая какой язык на какое место постпаить

скорее всего это будет lang1 lang2 lang3 : базовый - куда лукап - таргет

In [10]:
import pandas as pd
from tqdm.auto import tqdm
import math


In [4]:
df_en_ru_fr = pd.read_csv("./reverse_connected_dicts/to_release/en_ru_fr_V1.csv")
df_ru_fr_en = pd.read_csv("./reverse_connected_dicts/to_release/ru_fr_en_v1.csv")


In [5]:
merge_reverse_en_ru_fr_vs_ru_fr_en = pd.read_csv("./reverse_connected_dicts/to_release/merge_reverse_en_ru_fr_vs_ru_fr_en.csv")
df_fr_en_ru_v1 = pd.read_csv("./reverse_connected_dicts/to_release/fr_en_ru_v1.csv")


In [7]:
three_steps_lemmas_dict = pd.read_csv("./reverse_connected_dicts/to_release/three_steps_dict.csv")
three_steps_lemmas_dict.head()

,Unnamed: 0,fr,en,ru,vote
0,0,joyeusement,gaily,весело,1
1,1,joyeusement,merrily,весело,1
2,2,joyeusement,merrily,счастливо,1
3,3,joyeusement,blithely,весело,1
4,4,joyeusement,cheerily,весело,1


In [8]:
en_ru_fr_wordforms = pd.read_csv("./reverse_connected_dicts/to_release/en_ru_fr_wordforms.csv")
en_ru_fr_wordforms.head()

,Unnamed: 0,en,ru,fr
0,0,Atlantiss,атлантисом,no_equality
1,1,Atlantiss,атлантису,no_equality
2,2,Atlantiss,атлантидой,no_equality
3,3,Atlantiss,atlantis,no_equality
4,4,warranted,заслуживает,mérite


In [2]:
def detect_nan(string):
    
    if 'float' in str(type(string)): return True
    try:
        isnan = math.isnan(string) 
        if isnan == True: return True
    
    except:
        return False

In [3]:
def get_df_from_dict(glob_dct, current_lang1, current_lang2, current_lang3):
    lang1_list = []
    lang2_list = []
    lang3_list = []
    votes = []

    for lang_1_word in glob_dct.keys():
        #print(lang_1_word, glob_dct[lang_1_word],'\n')
        for lang_2_word in glob_dct[lang_1_word].keys():
            #print(lang_2_word, glob_dct[lang_1_word],'\n')
            for lang_3_word in glob_dct[lang_1_word][lang_2_word].keys():
                #print(lang_3_word, glob_dct[lang_1_word][lang_2_word])
                lang1_list.append(lang_1_word)
                lang2_list.append(lang_2_word)
                lang3_list.append(lang_3_word)
                votes.append(glob_dct[lang_1_word][lang_2_word][lang_3_word])
    data = pd.DataFrame(list(zip(lang1_list, lang2_list,lang3_list, votes)),
                        columns =[current_lang1, current_lang2, current_lang3,'vote'])
    return data

In [11]:
def merge_cycle_dicts(overall_df_and_new_df_list, lookup_from, lookup_to, target):
    reversed_dict = {}
    count = 0 
    for df in overall_df_and_new_df_list:
        if 'vote' not in list(df.columns): 
            df['vote'] = [1 for i in range(len(df))]
        for index, row in tqdm(df.iterrows(), total=df.shape[0]):
            #print(row['en'], row['ru'],row['fr'] )
            lang_one_word = row[lookup_from]
            lang_two_word = row[lookup_to]
            lang_three_word = row[target]
            vote = row['vote']
            #print(count, lang_one_word,lang_two_word,  lang_three_word, vote)
            isnan = False
            word_list = [lang_one_word,lang_two_word,  lang_three_word]
            for word in word_list:
                isnan = detect_nan(word)
                if isnan == True:
                    break
            if isnan == False:
                if lang_one_word in reversed_dict :
                    if lang_two_word in reversed_dict[lang_one_word]:
                        if lang_three_word in reversed_dict[lang_one_word][lang_two_word]:
                            #count_element = lang_one_word + '_' + lang_two_word + '_' + lang_three_word
                            reversed_dict[lang_one_word][lang_two_word][lang_three_word] += int(vote) 
                        else:
                            reversed_dict[lang_one_word][lang_two_word][lang_three_word] = int(vote) 
                    else:
                        reversed_dict[lang_one_word][lang_two_word] = {}
                        reversed_dict[lang_one_word][lang_two_word][lang_three_word] = int(vote) 
                else:
                    reversed_dict[lang_one_word] = {}
                    reversed_dict[lang_one_word][lang_two_word] = {}
                    reversed_dict[lang_one_word][lang_two_word][lang_three_word] = int(vote) 
        count += 1
            
    return reversed_dict
#gl_dct = merge_cycle_dicts([df_fr_en_ru, df_en_fr_ru], "fr", "en", "ru")
#gl_dct = merge_cycle_dicts([df_en_ru_fr, df_ru_fr_en], "ru", "fr", "en")
#gl_dct = merge_cycle_dicts([merge_reverse_en_ru_fr_vs_ru_fr_en, df_fr_en_ru_v1],  "fr", "en", "ru")
gl_dct = merge_cycle_dicts([three_steps_lemmas_dict, en_ru_fr_wordforms],"en", "ru","fr")



  3%|▎         | 10316/341086 [00:18<01:35, 3470.00it/s]

In [12]:
gl_dct

{'gaily': {'весело': {'joyeusement': 2}, 'ярко': {'no_equality': 2}},
 'merrily': {'весело': {'joyeusement': 2},
  'счастливо': {'joyeusement': 2},
  'тонет': {'no_equality': 1},
  'меррили': {'no_equality': 1},
  'поищи': {'no_equality': 1},
  'баю': {'no_equality': 1}},
 'blithely': {'весело': {'joyeusement': 2},
  'беспечно': {'allègrement': 3, 'avec insouciance': 1},
  'no_equality': {'allègrement': 1}},
 'cheerily': {'весело': {'joyeusement': 2}},
 'no_equality': {'радостно': {'joyeusement': 1,
   'allègrement': 1,
   'avec joie': 1,
   'en liesse': 1},
  'навеселе': {'gai': 1, 'entre deux vins': 1},
  'весело': {'pas triste': 1},
  'исключительно': {'pas triste': 1,
   'super': 1,
   'effroyablement': 1,
   'remarquablement': 1,
   'comme tout': 1,
   'exclusif': 1},
  'бодро': {'vivement': 1,
   'vigoureusement': 1,
   'allègrement': 1,
   'avec entrain': 1},
  'живо': {'clairement': 1, 'vivant': 1},
  'явственно': {'clairement': 1},
  'доходчиво': {'clairement': 1, 'lucidement'

In [13]:
three_steps_with_wordforms = get_df_from_dict(gl_dct, "en", "ru","fr")

In [15]:
three_steps_with_wordforms.to_csv("./reverse_connected_dicts/to_release/three_steps_with_wordforms.csv", index = False)

In [13]:
df_step_1.to_csv("./reverse_connected_dicts/to_release/three_steps_dict.csv")

In [12]:
#df_step_1 = get_df_from_dict(gl_dct, "fr", "en", "ru")
df_step_1 = get_df_from_dict(gl_dct, "ru", "fr", "en")

In [13]:
df_step_1.head()

,ru,fr,en,vote
0,весело,joyeusement,gaily,1
1,весело,joyeusement,merrily,1
2,весело,joyeusement,blithely,1
3,весело,joyeusement,cheerily,1
4,весело,gai,jolly,1


In [14]:
df_step_1.to_csv("./reverse_connected_dicts/to_release/merge_reverse_en_ru_fr_vs_ru_fr_en.csv")

In [14]:
gl_dct_2 = merge_cycle_dicts([df_step_1, df_en_ru_fr], "en","ru", "fr")

100%|██████████| 1685/1685 [00:00<00:00, 2973.68it/s]


In [16]:
df_after_step_2 = get_df_from_dict(gl_dct_2, "en","ru", "fr")


In [17]:
#df_fr_ru_en
gl_dct_3 = merge_cycle_dicts([df_after_step_2, df_fr_ru_en], "fr","ru","en" )

100%|██████████| 8385/8385 [00:01<00:00, 5091.12it/s]


In [19]:
df_after_step_3 = get_df_from_dict(gl_dct_3, "fr","ru","en" )

In [20]:
df_after_step_3.head()

,fr,ru,en,vote
0,tomber,упасть,fall,4
1,tomber,упасть,fell,1
2,tomber,упасть,fall down,2
3,tomber,упасть,drop,1
4,tomber,падать,fall,2


In [21]:
gl_dct_4 = merge_cycle_dicts([df_after_step_3, df_ru_fr_en], "ru","fr","en" )

100%|██████████| 36425/36425 [00:05<00:00, 6254.87it/s]


In [22]:
df_after_step_4 = get_df_from_dict(gl_dct_4, "ru","fr","en")


In [23]:
df_after_step_4.head(20)

,ru,fr,en,vote
0,упасть,tomber,fall,5
1,упасть,tomber,fell,1
2,упасть,tomber,fall down,3
3,упасть,tomber,drop,2
4,упасть,baisser,fall,1
5,упасть,baisser,drop,1
6,упасть,se laisser tomber,no_equality,1
7,упасть,faire chute,fall,1
8,упасть,faire chute,drop,1
9,упасть,faire chute,collapse,1


In [108]:
df_after_step_4.to_csv("/Users/nigula/LL/adjust_unigr_dict/reverse_connected_dicts/glob_dict_v2.csv")

# 4DIRECTIONS_ONLY

# 1шаг - два максимально родственных языка пополняют таргетный язык

In [27]:
df_fr_en_ru = pd.read_csv("/Users/nigula/LL/adjust_unigr_dict/reverse_connected_dicts/y_crossrevers_vs_pos_cycle/fr_en_ru_no_cossim.csv")
df_en_fr_ru = pd.read_csv("/Users/nigula/LL/adjust_unigr_dict/reverse_connected_dicts/y_crossrevers_vs_pos_cycle/en_fr_ru_no_cossim.csv")



In [28]:
gl_dct = merge_cycle_dicts([df_fr_en_ru, df_en_fr_ru], "fr", "en", "ru")

100%|██████████| 920/920 [00:00<00:00, 5098.00it/s]


In [30]:
df_after_step_1 = get_df_from_dict(gl_dct, "fr", "en", "ru")

In [31]:
df_en_ru_fr = pd.read_csv("/Users/nigula/LL/adjust_unigr_dict/reverse_connected_dicts/y_crossrevers_vs_pos_cycle/en_ru_fr_no_cossim.csv")

# 2 шаг - таргетный язык в паре с превым языком пополняет второй

In [32]:
gl_dct_2 = merge_cycle_dicts([df_after_step_1, df_en_ru_fr], "en","ru", "fr")

100%|██████████| 1685/1685 [00:00<00:00, 4654.70it/s]


In [33]:
df_after_step_2 = get_df_from_dict(gl_dct_2, "en","ru", "fr")

# 3 шаг - таргетный язык в паре со вторым языком пополняет первый

In [35]:
gl_dct_3 = merge_cycle_dicts([df_after_step_2, df_fr_ru_en], "fr","ru","en" )

100%|██████████| 8385/8385 [00:01<00:00, 4628.95it/s]


In [36]:
df_after_step_3 = get_df_from_dict(gl_dct_3, "fr","ru","en" )

In [37]:
df_after_step_3.head()

,fr,ru,en,vote
0,tomber,упасть,fall,4
1,tomber,упасть,fell,1
2,tomber,упасть,fall down,2
3,tomber,упасть,drop,1
4,tomber,падать,fall,2


In [39]:
df_after_step_3.to_csv("/Users/nigula/Desktop/adjust_unigr_dict/reverse_connected_dicts/y_crossrevers_vs_pos_cycle/three_steps_dict_v3.csv")

